# Download the non-blind ICLR2025 data from OpenReview

In [1]:
import numpy as np
import pandas as pd
import pylab as plt

import requests
import time

In [2]:
# Load iclr2025v1

iclr = pd.read_parquet('../data/iclr25v1.parquet')

iclr

,year,id,title,abstract,authors,decision,scores,keywords,labels
0,2017,B1-Hhnslg,Prototypical Networks for Few-shot Learning,A recent approach to few-shot classification c...,"Jake Snell, Kevin Swersky, Richard Zemel",Reject,"[6, 4, 5]","[deep learning, transfer learning]",transfer learning
1,2017,B1-q5Pqxl,Machine Comprehension Using Match-LSTM and Ans...,Machine comprehension of text is an important ...,"Shuohang Wang, Jing Jiang",Accept (Poster),"[6, 6, 7]","[natural language processing, deep learning]",language models
2,2017,B16Jem9xe,Learning in Implicit Generative Models,Generative adversarial networks (GANs) provide...,"Shakir Mohamed, Balaji Lakshminarayanan",Invite to Workshop Track,"[8, 7, 6]",[unsupervised learning],unlabeled
3,2017,B16dGcqlx,Third Person Imitation Learning,Reinforcement learning (RL) makes it possible ...,"Bradly C Stadie, Pieter Abbeel, Ilya Sutskever",Accept (Poster),"[6, 5, 6]",[],unlabeled
4,2017,B184E5qee,Improving Neural Language Models with a Contin...,We propose an extension to neural network lang...,"Edouard Grave, Armand Joulin, Nicolas Usunier",Accept (Poster),"[7, 9, 5]",[natural language processing],language models
...,...,...,...,...,...,...,...,...,...
34519,2025,zxO4WuVGns,Inverse decision-making using neural amortized...,Bayesian observer and actor models have provid...,,,[],"[bayesian actor models, perception and action,...",unlabeled
34520,2025,zxbQLztmwb,Emergent Symbol-Like Number Variables in Artif...,"Symbolic programs, defined by discrete variabl...",,,[],"[mechanistic interpretability, numeric cogniti...",unlabeled
34521,2025,zxqdVo9FjY,Generalization for Least Squares Regression wi...,Random matrix theory has proven to be a valuab...,,,[],"[generalization, random matrix theory, spiked ...",unlabeled
34522,2025,zyGrziIVdE,Exploration by Running Away from the Past,The ability to explore efficiently and effecti...,,,[],"[reinforcement learning, exploration, deep lea...",RL


In [3]:
%%time

# Query the accept/reject decisions and scores. Warning: TAKES MANY HOURS
# API cuts you off every 60 queries, then the code sleeps for 30 seconds

for num, forum_id in enumerate(iclr.id):
    if iclr.year[num] < 2025:
        continue
    
    if (num + 1) % 1000 == 0:
        print('*', end='')
    elif (num + 1) % 100 == 0:
        print('.', end='')

    year = iclr.year[num]
    
    forum_url = f'https://api2.openreview.net/notes?forum={forum_id}'
        
    json = requests.get(forum_url).json()
    
    if 'name' in json and json['name'] == 'RateLimitError':
        time.sleep(30)
        json = requests.get(forum_url).json()
        
    found_authors = False
    for i in range(len(json['notes'])):
        if 'authors' in json['notes'][i]['content']:
            authors = json['notes'][i]['content']['authors']['value']
            authors = ', '.join(authors)
            title = json['notes'][-1]['content']['title']['value']
            abstract = json['notes'][-1]['content']['abstract']['value']
            keywords = json['notes'][-1]['content']['keywords']['value']
            found_authors = True
            break
    if found_authors:
        iclr.at[num, 'authors'] = authors
        iclr.at[num, 'title'] = title
        iclr.at[num, 'abstract'] = abstract
        iclr.at[num, 'keywords'] = keywords
    else:
        print(f'No authors found: {num}, {forum_id}')

    found_decision = False
    for i in range(len(json['notes'])):
        if 'decision' in json['notes'][i]['content']:
            decision = json['notes'][i]['content']['decision']['value']
            found_decision = True
            break
        if 'withdrawal_confirmation' in json['notes'][i]['content']:
            decision = 'Withdrawn'
            found_decision = True
            break
        if 'desk_reject_comments' in json['notes'][i]['content']:
            decision = 'Desk rejected'
            found_decision = True
            break
    if found_decision:
        iclr.at[num, 'decision'] = decision
    else:
        print(f'No decision found: {num}, {forum_id}')
        
    scores = []
    for i in range(len(json['notes'])):
        if 'rating' in json['notes'][i]['content']:
            score = int(json['notes'][i]['content']['rating']['value'])
            scores.append(score)
    iclr.at[num, 'scores'] = scores

print('')

iclr.to_parquet('../data/iclr25v2.parquet')

.....*.........*.........*.........*.........*.........*.........*.........*.........*.........*.....
CPU times: user 5min 36s, sys: 11.3 s, total: 5min 47s
Wall time: 3h 47min 25s


In [4]:
iclr

,year,id,title,abstract,authors,decision,scores,keywords,labels
0,2017,B1-Hhnslg,Prototypical Networks for Few-shot Learning,A recent approach to few-shot classification c...,"Jake Snell, Kevin Swersky, Richard Zemel",Reject,"[6, 4, 5]","[deep learning, transfer learning]",transfer learning
1,2017,B1-q5Pqxl,Machine Comprehension Using Match-LSTM and Ans...,Machine comprehension of text is an important ...,"Shuohang Wang, Jing Jiang",Accept (Poster),"[6, 6, 7]","[natural language processing, deep learning]",language models
2,2017,B16Jem9xe,Learning in Implicit Generative Models,Generative adversarial networks (GANs) provide...,"Shakir Mohamed, Balaji Lakshminarayanan",Invite to Workshop Track,"[8, 7, 6]",[unsupervised learning],unlabeled
3,2017,B16dGcqlx,Third Person Imitation Learning,Reinforcement learning (RL) makes it possible ...,"Bradly C Stadie, Pieter Abbeel, Ilya Sutskever",Accept (Poster),"[6, 5, 6]",[],unlabeled
4,2017,B184E5qee,Improving Neural Language Models with a Contin...,We propose an extension to neural network lang...,"Edouard Grave, Armand Joulin, Nicolas Usunier",Accept (Poster),"[7, 9, 5]",[natural language processing],language models
...,...,...,...,...,...,...,...,...,...
34519,2025,zxO4WuVGns,Inverse decision-making using neural amortized...,Bayesian observer and actor models have provid...,"Dominik Straub, Tobias F. Niehues, Jan Peters,...",Accept (Poster),"[6, 6, 6]","[Bayesian actor models, perception and action,...",unlabeled
34520,2025,zxbQLztmwb,Emergent Symbol-Like Number Variables in Artif...,There is an open question of what types of num...,"Satchel Grant, Noah Goodman, James Lloyd McCle...",Reject,"[3, 5, 6, 5]","[mechanistic interpretability, numeric cogniti...",unlabeled
34521,2025,zxqdVo9FjY,Generalization for Least Squares Regression wi...,Random matrix theory has proven to be a valuab...,"Jiping Li, Rishi Sonthalia",Reject,"[5, 3, 5, 5, 6]","[Generalization, Random Matrix Theory, Spiked ...",unlabeled
34522,2025,zyGrziIVdE,Exploration by Running Away from the Past,The ability to explore efficiently and effecti...,"Paul-Antoine LE TOLGUENEC, Yann Besse, Florent...",Reject,"[3, 3, 5, 3]","[Reinforcement Learning, Exploration, Deep Lea...",RL
